In [1]:
pip install --quiet ipython-sql

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window as W

In [3]:
spark = (SparkSession.builder
        .master("local")
        .appName("PysparkSQL_SP")
        .config("spark.ui.port", "4050")
        .getOrCreate()
        )

In [4]:
df = (spark.read.format("parquet")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("gs://projeto_final_2021/parquet_to_bq/MG/part-00000-9d8558ed-a2f8-4115-ba0f-29a4713d8605-c000.snappy.parquet")
  .createOrReplaceTempView("mg")
     )

In [5]:
# Consulta da tabela inteira 
spark.sql("SELECT * FROM mg").show()

+--------------------+--------+--------------------+------------+--------------+----+---+------+
|                nome|situacao|               cargo|salarioBruto|salarioLiquido| ano|mes|estado|
+--------------------+--------+--------------------+------------+--------------+----+---+------+
|   AARAO LOPES COSTA|   ATIVO|                CABO|      5692.0|       4668.11|2020|  1|    MG|
|AARAO MARQUES DA ...|   ATIVO|AGENTE DE SEGURAN...|      4778.5|       4117.18|2020|  1|    MG|
|  AARON DUARTE DALLA|   ATIVO|ESPEC.EM POLITICA...|     20490.7|      16217.44|2020|  1|    MG|
|AARON FERRAREZ BO...|   ATIVO|             SD 1 CL|     4508.26|       3728.25|2020|  1|    MG|
|AARON FRANCA TEOFILO|   ATIVO|PROFESSOR DE EDUC...|     3124.36|       3875.45|2020|  1|    MG|
|ABADIA APARECIDA ...|   ATIVO|AUXILIAR DE SERVI...|     1091.13|       1338.45|2020|  1|    MG|
|ABADIA APARECIDA ...|   ATIVO|AUXILIAR DE SERVI...|     1128.76|       1384.61|2020|  1|    MG|
|ABADIA APARECIDA ...|   ATIVO

In [6]:
# Descrição de dados da tabela 
spark.sql('DESCRIBE MG').show(30)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|          nome|   string|   null|
|      situacao|   string|   null|
|         cargo|   string|   null|
|  salarioBruto|   double|   null|
|salarioLiquido|   double|   null|
|           ano|   bigint|   null|
|           mes|   bigint|   null|
|        estado|   string|   null|
+--------------+---------+-------+



In [15]:
#Quantidade de servidores por cargo

spark.sql('''SELECT cargo, COUNT(nome) AS Qtd_servidores FROM mg GROUP BY cargo''').show()

+--------------------+--------------+
|               cargo|Qtd_servidores|
+--------------------+--------------+
|TECNICO DE GESTAO...|           413|
|ANALISTA DE DESEN...|           256|
|PROFESSOR DE EDUC...|         37038|
|          3 SARGENTO|         67307|
|ASSISTENTE ADMINI...|         17864|
|             SD 2 CL|         13209|
|CONTRATO AGENTE D...|          9449|
|ANALISTA DE GESTA...|            36|
|ANALISTA DA POLIC...|          5749|
|MUSICO INSTRUMENT...|          1092|
|CONT. ADM. PROF. ...|           252|
|AUXILIAR DE APOIO...|         13918|
|      ANALISTA DE TV|          1389|
|AUXILIAR DE SERVI...|          1903|
|MEDICO DA AREA DE...|          1063|
|CONTRATO ASSISTEN...|          3298|
|TECNICO DE DESENV...|           644|
|TECNICO DE GESTAO...|           139|
|           MOTORISTA|            14|
|CONTRATO ADM MEDI...|           627|
+--------------------+--------------+
only showing top 20 rows



In [16]:
# Verificar ano, cargo e salario maior que 10000 no ano de 2021
spark.sql("""
SELECT
    ano, cargo, salarioLiquido 
FROM 
    mg
WHERE
    salarioLiquido > 10000 AND ano = 2021

""").show(30)


+----+--------------------+--------------+
| ano|               cargo|salarioLiquido|
+----+--------------------+--------------+
|2021|ESPEC.EM POLITICA...|      16523.95|
|2021|ESCRIVAO DE POLIC...|      10113.15|
|2021| ADVOGADO AUTARQUICO|      18800.38|
|2021|INVESTIGADOR DE P...|      12615.31|
|2021|               MAJOR|       13064.1|
|2021|             CAPITAO|      11304.24|
|2021|     PERITO CRIMINAL|      12056.75|
|2021|          1 SARGENTO|      16465.45|
|2021|               MAJOR|      15408.22|
|2021|PROFESSOR DE EDUC...|      12855.11|
|2021|          2 SARGENTO|      11479.38|
|2021|          3 SARGENTO|      10534.12|
|2021|INVESTIGADOR DE P...|      18192.64|
|2021|             CAPITAO|      12217.27|
|2021| DELEGADO DE POLICIA|      21456.92|
|2021|          2 SARGENTO|      11179.86|
|2021|AGENTE DE SEGURAN...|       11964.2|
|2021|ESCRIVAO DE POLIC...|      14187.42|
|2021|             CAPITAO|      21007.15|
|2021|          1 SARGENTO|      12179.79|
|2021|INVES

In [18]:
#Média salarial dos servidores no cargo de professor durante o decorrer do ano de 2020
spark.sql(
"""
SELECT
    cargo,
    ROUND(AVG(salarioLiquido),2) AS media_salario,
    ano,
    mes 
FROM
    mg
WHERE
    cargo LIKE "%PROF%" AND ano == 2020
GROUP BY
    mes, ano, cargo
ORDER BY
    mes
"""
).show(truncate=50)

+--------------------------------------------------+-------------+----+---+
|                                             cargo|media_salario| ano|mes|
+--------------------------------------------------+-------------+----+---+
|           PROFESSOR DE ENSINO MEDIO E TECNOLOGICO|      1264.49|2020|  1|
|                      PROFESSOR DE ARTE E RESTAURO|      1713.86|2020|  1|
|                      PROFESSOR DE EDUCACAO BASICA|      2383.58|2020|  1|
|                        PROFISSIONAL DE ENFERMAGEM|       4062.5|2020|  1|
|            CONT. ADM. PROF. DE ENSI. MEDIO E TEC.|       456.49|2020|  1|
|                               PROFESSOR DESIGNADO|       984.52|2020|  1|
|                                 PROFESSOR DE ARTE|      1841.58|2020|  1|
|                         PROFESSOR DE EDUCACAO BAS|      2935.83|2020|  1|
|                                         PROFESSOR|      1431.83|2020|  1|
|                    PROFESSOR DE EDUCACAO SUPERIOR|      5370.87|2020|  1|
|CONTRATO PR

In [19]:
# Verificar a quantidade de pagamento efetuado para cada servidor 
spark.sql('''
            SELECT 
                nome,
            COUNT
                (salarioLiquido) 
            FROM 
                mg 
            GROUP BY 
                nome 
            ''').show(10)

+--------------------+---------------------+
|                nome|count(salarioLiquido)|
+--------------------+---------------------+
|ABADIA APARECIDA ...|                   20|
|  ABILIO MIGUEL NETO|                   20|
|ACACIA DE PAULA C...|                   38|
|ADALBERTO CARLOS ...|                   20|
|ADAMO SOUZA NASCI...|                   20|
|ADELANIA DOS REIS...|                   20|
|ADELIA FELICIO GO...|                   20|
|ADELSON BARBOSA A...|                   20|
|ADENICIO FELISBER...|                   20|
|ADESIO MESQUITA B...|                   20|
+--------------------+---------------------+
only showing top 10 rows



In [7]:
#Top 5 menores médias salariais por cargo 
spark.sql("SELECT cargo, ROUND(AVG(salarioLiquido),2) AS media_salario_cargo FROM mg GROUP BY cargo ORDER BY media_salario_cargo limit 5").show() 

+--------------------+-------------------+
|               cargo|media_salario_cargo|
+--------------------+-------------------+
|           MOTORISTA|             571.48|
|CONT. ADM. PROF. ...|             853.63|
|    SERVENTE-ESCOLAR|              902.7|
|MINAS CAIXA - APO...|             906.15|
|GRATIF. ENCARGO C...|             979.83|
+--------------------+-------------------+



In [8]:
#Descobrir o número total de registros de cada ano
spark.sql("select ano, count(situacao) from mg group by ano").show() 

+----+---------------+
| ano|count(situacao)|
+----+---------------+
|2021|        2621814|
|2020|        4000943|
+----+---------------+



In [9]:
#Descobrir o número total de registros de cada ano de servidores ativos
spark.sql(
"""
SELECT 
    ano, count(situacao)
FROM 
    mg 
WHERE 
    situacao LIKE 'ATIVO'
GROUP BY 
    ano
"""
         ).show()

+----+---------------+
| ano|count(situacao)|
+----+---------------+
|2021|        2556483|
|2020|        3881693|
+----+---------------+



In [10]:
spark.sql("SELECT DISTINCT(NOME) AS contagem_nomes_distintos FROM mg").show(5) 

+------------------------+
|contagem_nomes_distintos|
+------------------------+
|    ABADIA APARECIDA ...|
|      ABILIO MIGUEL NETO|
|    ACACIA DE PAULA C...|
|    ADALBERTO CARLOS ...|
|    ADAMO SOUZA NASCI...|
+------------------------+
only showing top 5 rows



In [11]:
spark.sql("SELECT nome,cargo,situacao,mes,ano FROM mg WHERE nome LIKE 'ABILIO MIGUEL NETO' AND ano == 2020").show()

+------------------+--------------------+--------+---+----+
|              nome|               cargo|situacao|mes| ano|
+------------------+--------------------+--------+---+----+
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  1|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  2|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  3|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  4|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  5|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  6|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  7|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  8|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO|  9|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO| 10|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO| 11|2020|
|ABILIO MIGUEL NETO|PROFESSOR DE EDUC...|   ATIVO| 12|2020|
+------------------+--------------------+--------+---+----+



In [12]:
spark.sql("SELECT COUNT(NOME) FROM mg").show() 

+-----------+
|count(NOME)|
+-----------+
|    6622757|
+-----------+



In [13]:
spark.sql(
"""
SELECT
    cargo,
    ROUND(AVG(salarioLiquido),2) AS media_salario,
    ano,
    mes 
FROM
    mg
WHERE
    cargo LIKE "%TELE%" AND ano = 2020
GROUP BY
    mes, ano, cargo
ORDER BY
    mes
"""
).show(truncate=50)

+---------------------------------------------+-------------+----+---+
|                                        cargo|media_salario| ano|mes|
+---------------------------------------------+-------------+----+---+
|                   GESTOR DE TELECOMUNICACOES|      5627.52|2020|  1|
|ASSISTENTE ADMINISTRATIVO DE TELECOMUNICACOES|      5399.94|2020|  1|
|  AUXILIAR ADMINISTRATIVO DE TELECOMUNICACOES|      3174.99|2020|  1|
|ASSISTENTE ADMINISTRATIVO DE TELECOMUNICACOES|      5800.09|2020|  2|
|  AUXILIAR ADMINISTRATIVO DE TELECOMUNICACOES|      2593.12|2020|  2|
|                   GESTOR DE TELECOMUNICACOES|      5223.89|2020|  2|
|ASSISTENTE ADMINISTRATIVO DE TELECOMUNICACOES|      5814.21|2020|  3|
|                   GESTOR DE TELECOMUNICACOES|      5541.95|2020|  3|
|  AUXILIAR ADMINISTRATIVO DE TELECOMUNICACOES|      2604.95|2020|  3|
|                   GESTOR DE TELECOMUNICACOES|      5310.77|2020|  4|
|ASSISTENTE ADMINISTRATIVO DE TELECOMUNICACOES|       5630.0|2020|  4|
|  AUX

In [20]:
# Verificar os cargos existentes
spark.sql(
'''
SELECT
    cargo
FROM
    mg
GROUP By
cargo

'''

).show()

+--------------------+
|               cargo|
+--------------------+
|TECNICO DE GESTAO...|
|ANALISTA DE DESEN...|
|PROFESSOR DE EDUC...|
|          3 SARGENTO|
|ASSISTENTE ADMINI...|
|             SD 2 CL|
|CONTRATO AGENTE D...|
|ANALISTA DE GESTA...|
|ANALISTA DA POLIC...|
|MUSICO INSTRUMENT...|
|CONT. ADM. PROF. ...|
|AUXILIAR DE APOIO...|
|      ANALISTA DE TV|
|AUXILIAR DE SERVI...|
|MEDICO DA AREA DE...|
|CONTRATO ASSISTEN...|
|TECNICO DE DESENV...|
|TECNICO DE GESTAO...|
|           MOTORISTA|
|CONTRATO ADM MEDI...|
+--------------------+
only showing top 20 rows



In [21]:
#Quantidade de servidores por cargo

spark.sql('''SELECT cargo, COUNT(nome) AS Qtd_servidores FROM mg GROUP BY cargo''').show()

+--------------------+--------------+
|               cargo|Qtd_servidores|
+--------------------+--------------+
|TECNICO DE GESTAO...|           413|
|ANALISTA DE DESEN...|           256|
|PROFESSOR DE EDUC...|         37038|
|          3 SARGENTO|         67307|
|ASSISTENTE ADMINI...|         17864|
|             SD 2 CL|         13209|
|CONTRATO AGENTE D...|          9449|
|ANALISTA DE GESTA...|            36|
|ANALISTA DA POLIC...|          5749|
|MUSICO INSTRUMENT...|          1092|
|CONT. ADM. PROF. ...|           252|
|AUXILIAR DE APOIO...|         13918|
|      ANALISTA DE TV|          1389|
|AUXILIAR DE SERVI...|          1903|
|MEDICO DA AREA DE...|          1063|
|CONTRATO ASSISTEN...|          3298|
|TECNICO DE DESENV...|           644|
|TECNICO DE GESTAO...|           139|
|           MOTORISTA|            14|
|CONTRATO ADM MEDI...|           627|
+--------------------+--------------+
only showing top 20 rows



In [22]:
# média salarial por estado dos servidores ativos 
spark.sql(
'''
SELECT 
    ROUND(AVG(salarioLiquido),2) AS media_salarial ,
    estado
FROM 
    mg
WHERE
    situacao = "ATIVO"
GROUP BY
    estado
'''
).show()

+--------------+------+
|media_salarial|estado|
+--------------+------+
|       3640.67|    MG|
+--------------+------+



In [23]:
#verificar dados no mês de fevereiro de 2021
spark.sql("SELECT * FROM mg WHERE mes = 2 AND ano = 2021").show()

+--------------------+--------+--------------------+------------+--------------+----+---+------+
|                nome|situacao|               cargo|salarioBruto|salarioLiquido| ano|mes|estado|
+--------------------+--------+--------------------+------------+--------------+----+---+------+
|   AARAO LOPES COSTA|   ATIVO|                CABO|     6431.96|       5199.32|2021|  2|    MG|
|AARAO MARQUES DA ...|   ATIVO|AGENTE DE SEGURAN...|     5564.72|       4646.91|2021|  2|    MG|
|AARON CROSARA MAG...|   ATIVO|PROFESSOR DE EDUC...|     1937.89|       2356.39|2021|  2|    MG|
|  AARON DUARTE DALLA|   ATIVO|ESPEC.EM POLITICA...|    21295.76|      16523.97|2021|  2|    MG|
|AARON FERRAREZ BO...|   ATIVO| SOLDADO DE 1 CLASSE|     5094.33|       4848.92|2021|  2|    MG|
|AARON FRANCA TEOFILO|   ATIVO|PROFESSOR DE EDUC...|     2412.49|       2168.93|2021|  2|    MG|
|ABADIA APARECIDA ...|   ATIVO|AUXILIAR DE SERVI...|     1128.76|       1386.06|2021|  2|    MG|
|ABADIA APARECIDA ...|   ATIVO

In [24]:
#Valores gastos com o sálario ao longo dos mês ao longo do ano de 2021
spark.sql(
'''
SELECT
    mes,
    ROUND(Sum(salarioLiquido),2) AS soma_salario
FROM
    mg
WHERE 
    ano = 2021
GROUP BY 
    mes
ORDER BY
    mes ASC
 
''').show()

+---+---------------+
|mes|   soma_salario|
+---+---------------+
|  1| 1.2400598493E9|
|  2|1.22365755559E9|
|  3| 1.0961267572E9|
|  4|1.26313728345E9|
|  5|1.15904614923E9|
|  6|1.15581799958E9|
|  7|1.22169791787E9|
|  8|1.17879162649E9|
+---+---------------+



In [25]:
# Média salarial dos servidores ativos no ano de 2021
spark.sql(
"""
SELECT
    cargo,
    ROUND(AVG(salarioLiquido),2) AS media_salario,
    ano,
    mes,
    situacao
FROM
    mg
WHERE
    situacao = "ATIVO" AND ano = 2021
GROUP BY
    mes, ano, cargo, situacao
ORDER BY
    mes
"""
).show()

+--------------------+-------------+----+---+--------+
|               cargo|media_salario| ano|mes|situacao|
+--------------------+-------------+----+---+--------+
|TECNICO DA INDUST...|      6769.74|2021|  1|   ATIVO|
|CONTRATO ANALISTA...|      5065.72|2021|  1|   ATIVO|
|MEDICO DA AREA DE...|      9874.66|2021|  1|   ATIVO|
|ANALISTA EXECUTIV...|      5439.28|2021|  1|   ATIVO|
|AUXILIAR DE HEMAT...|      4744.97|2021|  1|   ATIVO|
|     AUDITOR INTERNO|     13378.33|2021|  1|   ATIVO|
|CONTRATO PROFISSI...|      4015.13|2021|  1|   ATIVO|
|TECNICO EM EDUCAC...|      3422.77|2021|  1|   ATIVO|
|             CAPITAO|     13492.51|2021|  1|   ATIVO|
|          3 SARGENTO|      6933.27|2021|  1|   ATIVO|
|      ANALISTA DE TV|      4000.38|2021|  1|   ATIVO|
|ESPECIALISTA EM E...|      3276.63|2021|  1|   ATIVO|
|FISCAL DE TRANSPO...|      6590.09|2021|  1|   ATIVO|
|CONTRATO ADMINIST...|      2179.09|2021|  1|   ATIVO|
|CONTRATO ADM. ANA...|      5016.56|2021|  1|   ATIVO|
|ANALISTA 